In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.request
import re
import time
from sqlalchemy import create_engine
import pickle
import os
from pathlib import Path
from psycopg2.errors import UndefinedTable
from webdriver_manager.chrome import ChromeDriverManager
#from webdriver_manager.firefox import GeckoDriverManager

In [2]:
class TimeLimitExpired(Exception):
    pass

def load_page(url, card_name, debug = False):
    '''
    setup - load entire page, pressing 'show more' button
    '''
    
    #fix card name
    card_name = card_name.replace('/', '-')
    
    #load debug file
    debug_path = Path(os.path.join(Path().absolute() , 'debug'))
    pickle_file = Path(os.path.join(debug_path , '%s.pickle'%card_name))
    if debug == True and pickle_file.is_file():
        html = None
        with open(pickle_file, 'rb') as file:
            html = pickle.load(file)
        return html
    
    driver_path = Path(os.path.join(Path().absolute(), 'chromedriver', 'chromedriver'))
    #driver = webdriver.Chrome(driver_path)
    driver = webdriver.Chrome('/usr/bin/chromedriver')
    #options = webdriver.ChromeOptions()
    #options.binary_location = '/opt/google/chrome/google-chrome'
    #service_log_path = "{}/chromedriver.log".format(logs_path)
    #service_args = ['--verbose']
    #driver = webdriver.Chrome(ChromeDriverManager().install(), #'/path/to/chromedriver',
    #        chrome_options=options)#,
            #service_args=service_args,
            #service_log_path=service_log_path)
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    #driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    delay = 2
    timeout = 0
    html = None
    try:
        while True:
            try:
                driver.find_element_by_xpath(r"//span[contains(text(),'Show more results')]").click()
                #reset timeout
                timeout = 0
                time.sleep(delay)
            except ElementClickInterceptedException as e:
                if timeout >= 60:
                    raise TimeLimitExpired('timeout >= 60')
                time.sleep(delay)
                timeout += delay
    except ElementNotInteractableException as e:
        #now we have all the data, SCRAPE IT!!!
        html = driver.page_source.encode('utf-8')
        
        #dump file for debug if no file exists
        if not debug_path.is_dir():
            os.makedirs(debug_path)
        if pickle_file.is_file() == False:
            with open(pickle_file, 'wb') as file:
                pickle.dump(html, file, protocol=pickle.HIGHEST_PROTOCOL)
    finally:
        driver.quit()
            
    return html

In [3]:
def get_soup(html):
    '''
    setup with selenium: get the html to scrape
    '''

    soup = BeautifulSoup(html, 'lxml')
    table_tag = soup.find_all('div', class_="table-body")
    table = table_tag[0]
    info_tag = soup.find_all('div', class_='tab-container d-flex flex-column h-100')
    info = info_tag[0]
    
    return info, table

In [4]:
def get_sales_available_items(row_tag):
    tag=row_tag.find_all('span', class_='badge badge-faded d-none d-sm-inline-flex has-content-centered mr-1 sell-count')[0]#.get_text().strip()
    regex = r'\d+\sSales\s|\s\d+\sAvailable\sitems'
    match = re.findall(regex, str(tag))
    sales = match[0][:-7]
    available_items = match[1][1:-16]
    return sales, available_items

def get_product_information(row_tag):
    tag=row_tag.find_all('div', class_='product-attributes col-auto col-md-12 col-xl-5')[0]#.get_text().strip()
    regex = r'showMsgBox\(this,\'.*?\'\)'
    match = re.findall(regex, str(tag))
    
    item_conditions = match[0][17:-2]
    item_languages = match[1][17:-2]
    
    item_is_playset = False
    for item in match:
        if item[17:-2] == 'Playset':
            item_is_playset = True

    item_is_foil = False
    for item in match:
        if item[17:-2] == 'Foil':
            item_is_foil = True
    return item_conditions, item_languages, item_is_playset, item_is_foil

def get_data(row_tags, card_name, debug=False):
    '''
    iterate through each row in the table, getting the data
    '''
    
    seller_names = []
    item_prices = []
    item_amounts = [] 
    seller_sales = []
    seller_available_items = []
    item_locations = []
    item_conditions = []
    item_languages = []
    item_is_playsets = []
    item_is_foils = []
    
    for row_tag in row_tags:
        seller_names.append(row_tag.find_all('span', class_='d-flex has-content-centered mr-1')[0].get_text().strip())

        item_prices.append(row_tag.find_all('span', class_='font-weight-bold color-primary small text-right text-nowrap')[0].get_text().strip()[:-2])

        item_amounts.append(row_tag.find_all('span', class_='item-count small text-right')[0].get_text().strip()[:])

        sales, available_items = get_sales_available_items(row_tag)
        seller_sales.append(sales)
        seller_available_items.append(available_items)

        item_locations.append(row_tag.find_all('span', class_='icon d-flex has-content-centered mr-1')[0]['title'][15:])

        item_condition, item_language, item_is_playset, item_is_foil= get_product_information(row_tag)
        item_conditions.append(item_condition)
        item_languages.append(item_language)
        item_is_playsets.append(item_is_playset)
        item_is_foils.append(item_is_foil)
    
    '''
    put it into pandas
    '''
    data_dict = {
        'card_name': [card_name for i in range(len(seller_names))],
        'ts': [pd.Timestamp.now(tz='UTC') for i in range(len(seller_names))],
        'list_order': [i for i in range(len(seller_names))], 
        'seller_name': seller_names,
        'seller_sales': seller_sales,
        'seller_available_items': seller_available_items,
        'item_price': item_prices,
        'item_amount': item_amounts,
        'item_location': item_locations,
        'item_condition': item_conditions,
        'item_language': item_languages,
        'item_is_playset': item_is_playsets,
        'item_is_foil': item_is_foils,
    }
    df = pd.DataFrame(data_dict)
    
    '''
    change some datatypes
    '''
    df.seller_sales = df.seller_sales.astype(int)
    df.seller_available_items = df.seller_available_items.astype(int)
    df.item_amount = df.item_amount.astype(int)
    df.item_price = df.item_price.str.replace(',', '.')
    df.item_price = df.item_price.astype(float)
    
    '''
    make some replacements
    '''
    df.item_condition = df.item_condition.replace(
        {'Mint': 'M', 'Near Mint': 'NM', 'Excellent': 'EX', 
         'Good': 'GD', 'Light Played': 'LP', 'Played': 'PL', 'Poor': 'P'})

    '''
    correct for playsets:
    '''
    df.loc[df.item_is_playset == True, 'item_price'] = \
        df.loc[df.item_is_playset == True, 'item_price'] / 4
    df.loc[df.item_is_playset == True, 'item_amount'] = \
        df.loc[df.item_is_playset == True, 'item_amount'] * 4

    if debug == True:
        display('seller_names', len(seller_names), seller_names[:10], 
            'item_prices', len(item_prices), item_prices[:10], 
            'item_amounts', len(item_amounts), item_amounts[:10], 
            'item_locations', len(item_locations), item_locations[:10], 
            'item_conditions', len(item_conditions), item_conditions[:10], 
            'item_languages', len(item_languages), item_languages[:10], 
            'item_is_playsets', len(item_is_playsets), 'True: %d, False: %d'%(len([i for i in item_is_playsets if item_is_playsets[i]==True]), len([i for i in item_is_playsets if item_is_playsets[i]==False])), 
            'item_is_foils', len(item_is_foils), 'True: %d, False: %d'%(len([i for i in item_is_foils if item_is_foils[i]==True]), len([i for i in item_is_foils if item_is_foils[i]==False])))
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
            display(df)
    return df

In [5]:
''' 
bitconnect to the database 
'''
def get_db_connection():
    username = 'mig'
    password = 'password' 
    host_name = 'localhost'
    port = 5432
    db_name = 'mtg'
    conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host_name, port, db_name)

    engine = create_engine(conn_str)
    return engine

In [6]:
def conditional_insert(engine, card_name, frequency=30, debug = False):
    '''
    Checks if its time to insert records in the database.
    We only want to insert records with a frequency of frequency.
    
    engine - the db engine
    card_name - the card name
    frequency - the frequency of db inserts
    '''
    
    now = pd.Timestamp.now(tz='UTC') #Timestamp('2019-10-09 15:09:44.173350+0000')
    minute = 0 if now.minute < 30 else 30
    now_date_time_hour = pd.Timestamp(now.year, now.month, now.day, now.hour, minute)
    
    #minus 1 minute to prevent conflicts with cron
    now_date_time_hour_puls_frequency_min = now_date_time_hour + pd.Timedelta('%d minutes'%(frequency - 1))
    
    query = '''
    SELECT COUNT(*)  
    FROM card_listings
    WHERE card_name = '%s' 
    AND ts::time BETWEEN '%s' AND '%s';
    '''%(card_name, now_date_time_hour, now_date_time_hour_puls_frequency_min)
    
    df_result = pd.read_sql_query(query, engine)
    
    if debug==True:
        print(query)
    
    return df_result.iloc[0][0], now_date_time_hour, now_date_time_hour_puls_frequency_min

In [7]:
def main(debug=False):
    '''
    the 6 debug files have 14.5 MB total
    14.5MB x 2 times per hour x 24 hours x 31 days = 21576 GB per month
    '''
    
    '''
    card_names and links
    '''
    card_names_urls = {
        'Snow Covered Island': 'https://www.cardmarket.com/en/Magic/Products/Singles/Modern-Horizons/Snow-Covered-Island', 
        'Fabled Passage': 'https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Fabled-Passage', 
        'Once Upon a Time': 'https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Once-Upon-a-Time', 
        'Murderous Rider // Swift End': 'https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Murderous-Rider-Swift-End', 
        'Questing Beast': 'https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Questing-Beast', 
        'Oko, Thief of Crowns': 'https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Oko-Thief-of-Crowns'
    }
    
    engine = get_db_connection()
    
    for card_name in card_names_urls:  
        
        '''
        checks if its time to insert data for this card, and skips it if its not
        '''
        count, start, end = conditional_insert(engine, card_name, frequency=30)
        if count > 0:
            print('There are already %d records from %s to %s'%(count, start, end))
            continue
        
        url = card_names_urls[card_name]
        html = load_page(url, card_name, debug=debug)
        info, table = get_soup(html)
        row_tags = table.find_all('div', class_='row no-gutters article-row')
        df = get_data(row_tags, card_name)
        
        print('inserting records of card %s with shape %s regarding time period between %s and %s'%(card_name, str(df.shape), str(start), str(end)))
        
        df.to_sql('card_listings', con=engine, if_exists='append', index=False)
        
        if debug == True:
            print(card_name)
            print(card_names_urls[card_name])
            print(df.shape)
            print(df.dtypes)
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
                display(df)
        
if __name__ == '__main__':
    main(debug=False)

inserting records of card Snow Covered Island with shape (1246, 13) regarding time period between 2019-10-10 22:00:00 and 2019-10-10 22:29:00
Snow Covered Island
https://www.cardmarket.com/en/Magic/Products/Singles/Modern-Horizons/Snow-Covered-Island
(1246, 13)
card_name                              object
ts                        datetime64[ns, UTC]
list_order                              int64
seller_name                            object
seller_sales                            int64
seller_available_items                  int64
item_price                            float64
item_amount                             int64
item_location                          object
item_condition                         object
item_language                          object
item_is_playset                          bool
item_is_foil                             bool
dtype: object


,card_name,ts,list_order,seller_name,seller_sales,seller_available_items,item_price,item_amount,item_location,item_condition,item_language,item_is_playset,item_is_foil
0,Snow Covered Island,2019-10-10 22:22:48.299834+00:00,0,toycastle,3355,119357,0.0900,1,,NM,English,False,False
1,Snow Covered Island,2019-10-10 22:22:48.299855+00:00,1,talcoon85,1223,14,0.0900,2,,NM,English,False,False
2,Snow Covered Island,2019-10-10 22:22:48.299860+00:00,2,greggoss,716,920,0.1000,1,,NM,French,False,False
3,Snow Covered Island,2019-10-10 22:22:48.299864+00:00,3,svilla92,282,24,0.1000,1,,NM,Spanish,False,False
4,Snow Covered Island,2019-10-10 22:22:48.299867+00:00,4,Soldatpacif,168,47,0.1000,1,,NM,French,False,False
5,Snow Covered Island,2019-10-10 22:22:48.299870+00:00,5,xFL0x,111,264,0.1000,1,,NM,German,False,False
6,Snow Covered Island,2019-10-10 22:22:48.299873+00:00,6,Gui-B,99,1659,0.1000,1,,NM,French,False,False
7,Snow Covered Island,2019-10-10 22:22:48.299876+00:00,7,arnaujuanco,50,238,0.1000,1,,NM,English,False,False
8,Snow Covered Island,2019-10-10 22:22:48.299880+00:00,8,Alphannibal,6,37,0.1000,1,,NM,French,False,False
9,Snow Covered Island,2019-10-10 22:22:48.299883+00:00,9,ERPIWI,2074,208,0.1100,1,,NM,English,False,False


inserting records of card Fabled Passage with shape (511, 13) regarding time period between 2019-10-10 22:00:00 and 2019-10-10 22:29:00
Fabled Passage
https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Fabled-Passage
(511, 13)
card_name                              object
ts                        datetime64[ns, UTC]
list_order                              int64
seller_name                            object
seller_sales                            int64
seller_available_items                  int64
item_price                            float64
item_amount                             int64
item_location                          object
item_condition                         object
item_language                          object
item_is_playset                          bool
item_is_foil                             bool
dtype: object


,card_name,ts,list_order,seller_name,seller_sales,seller_available_items,item_price,item_amount,item_location,item_condition,item_language,item_is_playset,item_is_foil
0,Fabled Passage,2019-10-10 22:22:50.397913+00:00,0,supersing,245,81,7.50,1,,NM,Spanish,False,False
1,Fabled Passage,2019-10-10 22:22:50.397936+00:00,1,Oppenheimer,40,834,7.95,1,,NM,German,False,False
2,Fabled Passage,2019-10-10 22:22:50.397941+00:00,2,AlphaCard,1074,2785,7.99,2,,NM,English,False,False
3,Fabled Passage,2019-10-10 22:22:50.397944+00:00,3,NerdyCaffe,246,219,8.00,1,,M,Italian,False,False
4,Fabled Passage,2019-10-10 22:22:50.397948+00:00,4,infrancesco,13,5,8.00,1,,NM,Italian,False,False
5,Fabled Passage,2019-10-10 22:22:50.397952+00:00,5,Angelus089,321,1097,8.10,1,,NM,Italian,False,False
6,Fabled Passage,2019-10-10 22:22:50.397955+00:00,6,MATZEBOBBEL666,166,695,8.46,1,,NM,German,False,False
7,Fabled Passage,2019-10-10 22:22:50.397959+00:00,7,matteonick,234,40,8.50,1,,NM,Spanish,False,False
8,Fabled Passage,2019-10-10 22:22:50.397962+00:00,8,Abbadon666,200,278,8.50,1,,NM,German,False,False
9,Fabled Passage,2019-10-10 22:22:50.397966+00:00,9,sonnywhite77,65,123,8.50,1,,NM,German,False,False


inserting records of card Once Upon a Time with shape (551, 13) regarding time period between 2019-10-10 22:00:00 and 2019-10-10 22:29:00
Once Upon a Time
https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Once-Upon-a-Time
(551, 13)
card_name                              object
ts                        datetime64[ns, UTC]
list_order                              int64
seller_name                            object
seller_sales                            int64
seller_available_items                  int64
item_price                            float64
item_amount                             int64
item_location                          object
item_condition                         object
item_language                          object
item_is_playset                          bool
item_is_foil                             bool
dtype: object


...

,card_name,ts,list_order,seller_name,seller_sales,seller_available_items,item_price,item_amount,item_location,item_condition,item_language,item_is_playset,item_is_foil
0,Murderous Rider // Swift End,2019-10-10 22:22:54.837050+00:00,0,Keika93,0,8,6.90,1,,M,German,False,False
1,Murderous Rider // Swift End,2019-10-10 22:22:54.837072+00:00,1,bluestream,52,229,7.00,1,,NM,German,False,False
2,Murderous Rider // Swift End,2019-10-10 22:22:54.837078+00:00,2,Jesusfiat,5,5,7.00,1,,NM,Spanish,False,False
3,Murderous Rider // Swift End,2019-10-10 22:22:54.837082+00:00,3,Voidgazer,9,7,7.09,1,,NM,German,False,False
4,Murderous Rider // Swift End,2019-10-10 22:22:54.837085+00:00,4,Fafari,24,3,7.10,1,,NM,German,False,False
5,Murderous Rider // Swift End,2019-10-10 22:22:54.837089+00:00,5,SarcoZQ,188,98,7.19,1,,NM,English,False,False
6,Murderous Rider // Swift End,2019-10-10 22:22:54.837092+00:00,6,SteGoesApex,14,10,7.20,1,,M,Italian,False,False
7,Murderous Rider // Swift End,2019-10-10 22:22:54.837096+00:00,7,Alexanderbg96,38,18,7.25,1,,M,English,False,False
8,Murderous Rider // Swift End,2019-10-10 22:22:54.837100+00:00,8,Daniele89x,32,2,7.30,1,,NM,Italian,False,False
9,Murderous Rider // Swift End,2019-10-10 22:22:54.837103+00:00,9,Jesszir,116,10,7.40,1,,M,English,False,False


inserting records of card Questing Beast with shape (489, 13) regarding time period between 2019-10-10 22:00:00 and 2019-10-10 22:29:00
Questing Beast
https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Questing-Beast
(489, 13)
card_name                              object
ts                        datetime64[ns, UTC]
list_order                              int64
seller_name                            object
seller_sales                            int64
seller_available_items                  int64
item_price                            float64
item_amount                             int64
item_location                          object
item_condition                         object
item_language                          object
item_is_playset                          bool
item_is_foil                             bool
dtype: object


,card_name,ts,list_order,seller_name,seller_sales,seller_available_items,item_price,item_amount,item_location,item_condition,item_language,item_is_playset,item_is_foil
0,Questing Beast,2019-10-10 22:22:56.572158+00:00,0,PkCore,0,97,12.4900,1,,M,Spanish,False,False
1,Questing Beast,2019-10-10 22:22:56.572179+00:00,1,mukz444,1,12,13.5000,1,,NM,German,False,False
2,Questing Beast,2019-10-10 22:22:56.572184+00:00,2,infrancesco,13,5,13.7000,1,,NM,Italian,False,False
3,Questing Beast,2019-10-10 22:22:56.572188+00:00,3,Kriloff,23,321,13.7500,2,,NM,English,False,False
4,Questing Beast,2019-10-10 22:22:56.572191+00:00,4,max1611,273,746,13.9900,1,,NM,Italian,False,False
5,Questing Beast,2019-10-10 22:22:56.572195+00:00,5,scicluna,1480,2712,14.0000,1,,NM,English,False,False
6,Questing Beast,2019-10-10 22:22:56.572198+00:00,6,Pedroc,997,143,14.0000,1,,NM,English,False,False
7,Questing Beast,2019-10-10 22:22:56.572201+00:00,7,Ludmilla,160,1301,14.0000,1,,NM,Italian,False,False
8,Questing Beast,2019-10-10 22:22:56.572204+00:00,8,sgtbeardy,64,59,14.0000,1,,NM,English,False,False
9,Questing Beast,2019-10-10 22:22:56.572207+00:00,9,Sunshine6992,56,151,14.0000,1,,NM,Spanish,False,False


inserting records of card Oko, Thief of Crowns with shape (439, 13) regarding time period between 2019-10-10 22:00:00 and 2019-10-10 22:29:00
Oko, Thief of Crowns
https://www.cardmarket.com/en/Magic/Products/Singles/Throne-of-Eldraine/Oko-Thief-of-Crowns
(439, 13)
card_name                              object
ts                        datetime64[ns, UTC]
list_order                              int64
seller_name                            object
seller_sales                            int64
seller_available_items                  int64
item_price                            float64
item_amount                             int64
item_location                          object
item_condition                         object
item_language                          object
item_is_playset                          bool
item_is_foil                             bool
dtype: object


,card_name,ts,list_order,seller_name,seller_sales,seller_available_items,item_price,item_amount,item_location,item_condition,item_language,item_is_playset,item_is_foil
0,"Oko, Thief of Crowns",2019-10-10 22:22:58.084172+00:00,0,mhskou,0,502,23.000,1,,NM,English,False,False
1,"Oko, Thief of Crowns",2019-10-10 22:22:58.084194+00:00,1,Sibarita,36,63,24.490,1,,M,Spanish,False,False
2,"Oko, Thief of Crowns",2019-10-10 22:22:58.084199+00:00,2,PkCore,0,97,24.490,1,,M,Spanish,False,False
3,"Oko, Thief of Crowns",2019-10-10 22:22:58.084202+00:00,3,Lander12345,31,716,25.500,1,,NM,Spanish,False,False
4,"Oko, Thief of Crowns",2019-10-10 22:22:58.084206+00:00,4,Pascalmoieem,1,58,25.500,1,,NM,English,False,False
5,"Oko, Thief of Crowns",2019-10-10 22:22:58.084209+00:00,5,Zeus54321,0,2,25.500,1,,NM,English,False,False
6,"Oko, Thief of Crowns",2019-10-10 22:22:58.084212+00:00,6,AlfaRuLeZ,1539,4702,25.990,1,,M,Spanish,False,False
7,"Oko, Thief of Crowns",2019-10-10 22:22:58.084216+00:00,7,SLAINETH,5138,57622,26.000,1,,NM,Spanish,False,False
8,"Oko, Thief of Crowns",2019-10-10 22:22:58.084219+00:00,8,MagicPal,924,585,26.000,1,,NM,Spanish,False,False
9,"Oko, Thief of Crowns",2019-10-10 22:22:58.084222+00:00,9,danymdc,215,478,26.000,1,,NM,Spanish,False,False


In [8]:
!jupyter nbconvert --to script prototype_scraping.ipynb

[NbConvertApp] Converting notebook prototype_scraping.ipynb to script
[NbConvertApp] Writing 12394 bytes to prototype_scraping.py
